<a href="https://colab.research.google.com/github/simulate111/Introduction-to-Human-Language-Technology/blob/main/Exercise%20task%201%3A%20lexical%20ambiguity(English-EWT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Libraries
import nltk
from nltk.tokenize import word_tokenize
import spacy
import re
nltk.download('punkt')
from collections import Counter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
!wget https://raw.githubusercontent.com/simulate111/Introduction-to-Human-Language-Technology/main/en_ewt-ud-train.conllu

--2024-03-16 00:31:52--  https://raw.githubusercontent.com/simulate111/Introduction-to-Human-Language-Technology/main/en_ewt-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13846707 (13M) [text/plain]
Saving to: ‘en_ewt-ud-train.conllu.2’

en_ewt-ud-train.con 100%[===================>]  13.21M  --.-KB/s    in 0.1s    

2024-03-16 00:31:52 (103 MB/s) - ‘en_ewt-ud-train.conllu.2’ saved [13846707/13846707]



In [3]:
#Load data and tokenize
with open("en_ewt-ud-train.conllu", "r") as file:
    data = file.read()

lines = data.split("\n")
tokens = []
for line in lines:
    parts = line.split("\t")
    try:
        tokens.append(parts[1])
    except IndexError:
        continue
text = " ".join(tokens)
tokens = word_tokenize(text)
print(tokens[:10])

['Al', '-', 'Zaman', ':', 'American', 'forces', 'killed', 'Shaikh', 'Abdullah', 'al']


In [4]:
text[:500]

'Al - Zaman : American forces killed Shaikh Abdullah al - Ani , the preacher at the mosque in the town of Qaim , near the Syrian border . [ This killing of a respected cleric will be causing us trouble for years to come . ] DPA : Iraqi authorities announced that they had busted up 3 terrorist cells operating in Baghdad . Two of them were being run by 2 officials of the Ministry of the Interior ! The MoI in Iraq is equivalent to the US FBI , so this would be like having J. Edgar Hoover unwittingly'

In [5]:
#Removing some common unnecessary symbols and characters
import re
tokenized = [re.sub(r'[^\w\s]', '', token) for token in tokens]
tokenized = [re.sub(r"(n't)", r" \1", token) for token in tokenized]
tokenized = [token for token in tokenized if not re.match(r'\d+', token)]
tokenized = [line for line in tokenized if line.strip()]
tokenized = [token for token in tokenized if len(token) > 1]
for line in tokenized[:10]:
    print(line)

Al
Zaman
American
forces
killed
Shaikh
Abdullah
al
Ani
the


In [6]:
num_words = len(tokenized)
print('THe total number of words in the dataset')
num_words

THe total number of words in the dataset


171448

In [7]:
# Count word frequencies
word_counts = Counter(tokenized)
sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
for word, count in sorted_word_counts[:10]:
    print(f"{word}: {count}")

the: 8151
to: 5077
and: 4855
of: 3589
in: 2911
is: 2178
you: 2029
that: 1981
for: 1789
it: 1588


**To identify common verbs and nouns, two dictionary of nouns and verbs are made and the common words in both dictionaries will be counted.
Language model of en_core_web_sm in soacy library is used as facilator.**

In [8]:
nlp = spacy.load("en_core_web_sm")
pattern = re.compile(r'[^\w\s]')
chunks = text.split('.')
chunks = [pattern.sub('', chunk) for chunk in chunks]
noun_freq = {}
verb_freq = {}

for chunk in chunks:
    doc = nlp(chunk)
    for token in doc:
        if len(token.text) == 1:
            continue
        if token.pos_ == 'NOUN':
            noun_freq[token.text] = noun_freq.get(token.text, 0) + 1
        elif token.pos_ == 'VERB':
            verb_freq[token.text] = verb_freq.get(token.text, 0) + 1

In [9]:
print("Total number of nouns:", sum(noun_freq.values()))
print("Total number of verbs:", sum(verb_freq.values()))

Total number of nouns: 32572
Total number of verbs: 23010


In [10]:
sorted_noun_freq = sorted(noun_freq.items(), key=lambda x: x[1], reverse=True)
print("Top 10 NOUN:")
for noun, freq in sorted_noun_freq[:10]:
    print(f"{noun}: {freq}")

sorted_verb_freq = sorted(verb_freq.items(), key=lambda x: x[1], reverse=True)
print("\nTop 10 VERB:")
for verb, freq in sorted_verb_freq[:10]:
    print(f"{verb}: {freq}")

Top 10 NOUN:
time: 386
people: 233
way: 194
place: 181
year: 168
day: 166
food: 158
service: 157
years: 154
Thanks: 147

Top 10 VERB:
have: 735
get: 354
know: 339
had: 316
do: 264
go: 260
said: 233
want: 208
going: 191
see: 185


In [11]:
common_words = set(noun_freq.keys()).intersection(set(verb_freq.keys()))
print("Number of common words between nouns and verbs:", len(common_words))

Number of common words between nouns and verbs: 798
